MONOMER GUI DETECTOR


In [2]:
import cv2
import numpy as np
import pandas as pd
import os
#import tqdm for jupyter notebook
from tqdm.notebook import tqdm, trange
from scipy.ndimage   import gaussian_filter

DATA_PATH = 'C:/Users/amityu/Gel_Sheet_Data/'
#DATA_PATH = r'D:\amityu\backoffice_data\\'

#movie = 'Control'
#movie ='140721'
#movie ='150721'
#movie ='100621'
#movie = '130721_CCA60_RAW'
#movie ='280523 AM100 568_2'
#movie ='280523 AM100 568_3'
#movie ='cca120'
#movie = 'control_1_050721'
#movie = 'control_2_230523'
#movie = 'cca120_am200'
#movie = 'cca40'
movie = 'cca300'
#movie = 'cca60_dist'
MOVIE_PATH = DATA_PATH +  movie + '/'
GRAPH_PATH = 'C:/Users/amityu/Gel_Sheet_Graph/'
#import gaussian from opencv
#from cv2 import GaussianBlur as gaussian

In [3]:
gel = np.load(MOVIE_PATH + 'np/gel_corrected.npy', mmap_mode='r')

In [1]:
del gel

NameError: name 'gel' is not defined

In [3]:
imgs_float = np.load(MOVIE_PATH + 'np/gel_corrected.npy').astype(np.float32) # replace with your images

imgs = np.zeros(imgs_float.shape, dtype='uint8')
imgs_float[np.isnan(imgs_float)] = 0
for t in trange(len(imgs_float)):
    img = imgs_float[t]
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    img = img.astype(np.uint8)
    imgs[t] = img

  0%|          | 0/57 [00:00<?, ?it/s]

In [4]:
save_data_file_name = MOVIE_PATH + 'np/monomer_rect_5.csv'

In [50]:
# Run at first time else deleted file
if ~os.path.isfile(save_data_file_name):
    df = pd.DataFrame()
    df['frame'] = np.arange(len(gel))
    df['Z'] = [0]* len(gel)
    df['Y'] = [50]* len(gel)
    df['X'] = [50]* len(gel)
    df['r_size'] = [15]* len(gel)
    df['y_gap'] = [20]* len(gel)  # scroll step in y
    df['mean'] = [0]* len(gel)
    df['std'] = [0]* len(gel)
    df['gap_from_surface'] = [10]* len(gel)
    #save dataframes
    df.to_csv(save_data_file_name, index=True)


In [5]:
import cv2
import numpy as np
global iy
# Global variables
frame_index = 0
enlarge_image_factor = 2
df = pd.read_csv(save_data_file_name, index_col=0)
df.set_index('frame', inplace=True)


# Define the draw_rectangle callback function
def draw_rectangle(event, x, z, flags, param):

    mean = df.loc[frame_index, 'mean']
    std = df.loc[frame_index, 'std']

    r_size= df.loc[frame_index, 'r_size']
    gap_from_surface = df.loc[frame_index, 'gap_from_surface']
    y_gap = df.loc[frame_index, 'y_gap']
    y = df.loc[frame_index, 'Y']
    if event == cv2.EVENT_LBUTTONDOWN:
        
        i_data = gel[frame_index, z + gap_from_surface:z + r_size, y:y + y_gap, x:x + r_size]
        i_data = i_data[~np.isnan(i_data)]
        mean = np.nanmean(i_data)
        std = np.nanstd(i_data)
        if np.isnan(mean):
            mean = 0
        if np.isnan(std):
            std = 0
        df.loc[frame_index] = pd.Series([z, y, x, r_size, y_gap, mean, std, gap_from_surface], index=df.columns)

    elif event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:  # scroll up
            if y + y_gap < imgs.shape[2]:
                y+= y_gap  # cycle through images

        else:  # scroll down
            y-= y_gap  # cycle through images
            if y< 0:
                y = 0
    #img_index =24
    img_no_nan = np.max(gel[frame_index, :, y:y + y_gap, :], axis=1)
    img_no_nan[np.isnan(img_no_nan)] = 0
    img_no_nan = img_no_nan.astype(np.int8)
    img_orig =cv2.convertScaleAbs(img_no_nan, alpha=1, beta=0)# load the new image

    img = img_orig.copy()

    cv2.rectangle(img, pt1=(x, z), pt2=(x+r_size, z+r_size), color= 1, thickness=2)
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    colored_image = cv2.applyColorMap(img, cv2.COLORMAP_JET)

    # Display the image
    cv2.imshow('image', colored_image)
    #cv2.imshow('image', img)

    data_string = f'Time: {frame_index}, X: {x}, Y: {y}, Z: {z} , Y MIP size: {y_gap}'
    data_string2 = f' , r_size: {r_size}, gap_from_surface: {gap_from_surface}'
    data_string3 = f' , mean: {mean:.2f}, std: {std:.2f}'
    exit_string = 's  to save, q to quit, 2/8 to change time'
    data_img = np.zeros((500,600))
    position = (10, 50)  # bottom-left corner of the text
    position_exit = (10, 100)  # bottom-left corner of the text
    position2 = (10, 150)  # bottom-left corner of the text
    position3 = (10, 200)  # bottom-left corner of the text

    fontScale = 0.7
    font = cv2.FONT_HERSHEY_SIMPLEX
    color = (255, 255, 255)  # white color
    thickness = 2  # line thickness
    cv2.putText(data_img, data_string,  position, font, fontScale, color, thickness)
    cv2.putText(data_img, exit_string, position_exit, font, fontScale, color, thickness)
    cv2.putText(data_img, data_string2, position2, font, fontScale, color, thickness)
    cv2.putText(data_img, data_string3, position3, font, fontScale, color, thickness)
    cv2.imshow('Data', data_img)
    return colored_image

# 3D array of images
img = np.zeros_like(gel[0, :, 0, :], dtype=np.uint8)

# Create the named window and set the mouse callback
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.namedWindow('Data', cv2.WINDOW_NORMAL)

cv2.setMouseCallback('image', draw_rectangle)
#cv2.setMouseCallback('Data', getData)


# Show the image and wait for the user to close the window
colored_image = cv2.applyColorMap(img, cv2.COLORMAP_JET)

while True:

    # Display the image
    cv2.imshow('image', colored_image)
    key = cv2.waitKey(1) & 0xFF
    if key  == ord('q'):
        cv2.imwrite(DATA_PATH + 'monomer_app.jpg', colored_image)

        break
    elif key == ord('2'):
        frame_index +=1
        if frame_index >= len(gel):
            frame_index = 0
        #y = df.iloc[frame_index].values[1].astype(int)
        colored_image = draw_rectangle(None, 0, 0, 1, 0)
    elif key == ord('8'):
        frame_index -=1
        if frame_index < 0:
            frame_index = len(imgs) - 1
        colored_image=draw_rectangle(None, 0, 0, -1, 0)
        #y = df.iloc[frame_index].values[1].astype(int)
    elif key == ord('s'):
        df.to_csv(save_data_file_name, index=True)

cv2.destroyAllWindows()

# Print the saved coordinates


C:\Users\amityu\AppData\Local\Temp\ipykernel_23732\581538910.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.044921875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[frame_index] = pd.Series([z, y, x, r_size, y_gap, mean, std, gap_from_surface], index=df.columns)
C:\Users\amityu\AppData\Local\Temp\ipykernel_23732\581538910.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.01556396484375' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[frame_index] = pd.Series([z, y, x, r_size, y_gap, mean, std, gap_from_surface], index=df.columns)


In [42]:
import cv2
import numpy as np


def create_image_text(text, height, width):
    image = np.zeros((height, width))
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Get the text size in pixels
    textSize = cv2.getTextSize(text, font, 1, 2)[0]

    # Get the location where the text needs to be put for center alignment
    textX = (image.shape[1] - textSize[0]) // 2
    textY = (image.shape[0] + textSize[1]) // 2

    # Add the text
    cv2.putText(image, text, (textX, textY), font, 1, (255, 255, 255), 2)

    return image


image = np.zeros((500,600))

# Ensure that text image and input image have the same width
text = create_image_text("Your text here", height=image.shape[0] // 2, width=image.shape[1])

combined_image = np.concatenate((image, text), axis=0)

cv2.imshow('Window', combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
df = pd.read_csv(MOVIE_PATH + 'np/monomer_rect.csv', index_col=0)
plt.errorbar(np.arange(len(df)), df['mean'], yerr=df['std'], fmt='o')
plt.xlabel('Time')
plt.ylabel('Intensity')
plt.title('Monomer intensity')
plt.show()      
